In [38]:
import codecs
import re
import pymorphy2
from nltk.corpus import stopwords
import time
import fileinput
from collections import Counter
from nltk.corpus import stopwords
import plotly as pt
import plotly.offline as offline
import plotly.graph_objs as go

In [44]:
main_lenta = "lenta-ru-news.csv"
for_count = 'words_count.txt'
counted = 'count.txt'
rus_stopwords = stopwords.words("russian")

# adding extended stop words from yandex direct
extended = ['', 'a', 'about', 'all', 'am', 'an', 'and', 'any', 'are', 'as', 'at', 'be', 'been', 'but', 'by', 'can', 'could', 'do', 'for', 'from', 'has', 'have', 'i', 'if', 'in', 'is', 'it', 'me', 'my', 'no', 'not', 'of', 'on', 'one', 'or', 'so', 'that', 'the', 'them', 'there', 'they', 'this', 'to', 'was', 'we', 'what', 'which', 'will', 'with', 'would', 'you', 'а', 'будем', 'будет', 'будете', 'будешь', 'буду', 'будут', 'будучи', 'будь', 'будьте', 'бы', 'был', 'была', 'были', 'было', 'быть', 'в', 'вам', 'вами', 'вас', 'весь', 'во', 'вот', 'все', 'всё', 'всего', 'всей', 'всем', 'всём', 'всеми', 'всему', 'всех', 'всею', 'всея', 'всю', 'вся', 'вы', 'да', 'для', 'до', 'его', 'едим', 'едят', 'ее', 'её', 'ей', 'ел', 'ела', 'ем', 'ему', 'емъ', 'если', 'ест', 'есть', 'ешь', 'еще', 'ещё', 'ею', 'же', 'за', 'и', 'из', 'или', 'им', 'ими', 'имъ', 'их', 'к', 'как', 'кем', 'ко', 'когда', 'кого', 'ком', 'кому', 'комья', 'которая', 'которого', 'которое', 'которой', 'котором', 'которому', 'которою', 'которую', 'которые', 'который', 'которым', 'которыми', 'которых', 'кто', 'меня', 'мне', 'мной', 'мною', 'мог', 'моги', 'могите', 'могла', 'могли', 'могло', 'могу', 'могут', 'мое', 'моё', 'моего', 'моей', 'моем', 'моём', 'моему', 'моею', 'можем', 'может', 'можете', 'можешь', 'мои', 'мой', 'моим', 'моими', 'моих', 'мочь', 'мою', 'моя', 'мы', 'на', 'нам', 'нами', 'нас', 'наса', 'наш', 'наша', 'наше', 'нашего', 'нашей', 'нашем', 'нашему', 'нашею', 'наши', 'нашим', 'нашими', 'наших', 'нашу', 'не', 'него', 'нее', 'неё', 'ней', 'нем', 'нём', 'нему', 'нет', 'нею', 'ним', 'ними', 'них', 'но', 'о', 'об', 'один', 'одна', 'одни', 'одним', 'одними', 'одних', 'одно', 'одного', 'одной', 'одном', 'одному', 'одною', 'одну', 'он', 'она', 'оне', 'они', 'оно', 'от', 'по', 'при', 'с', 'сам', 'сама', 'сами', 'самим', 'самими', 'самих', 'само', 'самого', 'самом', 'самому', 'саму', 'свое', 'своё', 'своего', 'своей', 'своем', 'своём', 'своему', 'своею', 'свои', 'свой', 'своим', 'своими', 'своих', 'свою', 'своя', 'себе', 'себя', 'собой', 'собою', 'та', 'так', 'такая', 'такие', 'таким', 'такими', 'таких', 'такого', 'такое', 'такой', 'таком', 'такому', 'такою', 'такую', 'те', 'тебе', 'тебя', 'тем', 'теми', 'тех', 'то', 'тобой', 'тобою', 'того', 'той', 'только', 'том', 'томах', 'тому', 'тот', 'тою', 'ту', 'ты', 'у', 'уже', 'чего', 'чем', 'чём', 'чему', 'что', 'чтобы', 'эта', 'эти', 'этим', 'этими', 'этих', 'это', 'этого', 'этой', 'этом', 'этому', 'этот', 'этою', 'эту', 'я', 'мені', 'наші', 'нашої', 'нашій', 'нашою', 'нашім', 'ті', 'тієї', 'тією', 'тії', 'те']
for _ in extended:
    rus_stopwords.append(_)

In [52]:
df = pd.read_csv(main_lenta)
df.shape

C:\Users\chdan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning:

Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.



(800975, 6)

In [53]:
df.head(10)

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18
5,https://lenta.ru/news/1999/08/31/stancia_mir/,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...,Россия,Все,1999/08/31
6,https://lenta.ru/news/1999/08/31/vzriv/,Взрыв в центре Москвы: пострадало 30 человек,В зале игровых автоматов в третьем ярусе подзе...,Россия,Все,1999/08/31
7,https://lenta.ru/news/1999/08/31/credit_japs/,Япония кредитует Россию на полтора миллиарда д...,Япония приняла решение разморозить кредиты Рос...,Россия,Все,1999/08/31
8,https://lenta.ru/news/1999/08/31/diana/,Британцы отмечают двухлетие смерти Дианы,Британцы отмечают сегодня скорбную дату - втор...,Мир,Все,1999/08/31
9,https://lenta.ru/news/1999/08/31/mvf/,Отмытые через Bank of NY деньги не имели отнош...,В понедельник директор департамента внешних св...,Россия,Все,1999/08/31


In [54]:
descr = df.describe()
descr

,url,title,text,topic,tags,date
count,800975,800975,800970,738973,773756,800975
unique,800964,797832,800037,23,94,7393
top,https://lenta.ru/news/2000/11/03/dagestan/,В Москве объявлено штормовое предупреждение,"РИА ""Новости""",Россия,Все,2019/12/05
freq,2,21,291,160445,453762,284


In [57]:
print("Уникальных статей по URL: {}, повторяющиеся {}".format(descr['url'].iloc[1], descr['url'].iloc[0]-descr['url'].iloc[1]))
print("Уникальных Заголовков статей: {}, повторяющиеся {}".format(descr['title'].iloc[1], descr['title'].iloc[0]-descr['title'].iloc[1]))
print("Уникальных текстов: {}, повторяющиеся {}".format(descr['text'].iloc[1], descr['text'].iloc[0]-descr['text'].iloc[1]))
print("Уникальных тем текстов: {}, повторяющиеся {}".format(descr['topic'].iloc[1], descr['topic'].iloc[0]-descr['topic'].iloc[1]))
print("Уникальных тагов по темам: {}, повторяющиеся {}".format(descr['tags'].iloc[1], descr['tags'].iloc[0]-descr['tags'].iloc[1]))

Уникальных статей по URL: 800964, повторяющиеся 11
Уникальных Заголовков статей: 797832, повторяющиеся 3143
Уникальных текстов: 800037, повторяющиеся 933
Уникальных тем текстов: 23, повторяющиеся 738950
Уникальных тагов по темам: 94, повторяющиеся 773662


### Подсчет статей по темам
выыведем таблицу с группировкой по топикам, чтобы просмотреть распределение статей по темам. Это поможет сделать вывод о направленности статей ( из наших данных видно, что в основномэто политические и экономические темы)

In [56]:
upd_df = df.groupby('topic').count().reset_index()
sort_df = upd_df.sort_values(['url']).tail(27)
sort_df

,topic,url,title,text,tags,date
11,МедНовости,1,1,1,1,1
18,Сочи,1,1,1,1,1
21,ЧМ-2014,2,2,2,2,2
14,Оружие,3,3,3,3,3
1,Библиотека,65,65,65,65,65
...,...,...,...,...,...,...
9,Культура,53797,53797,53796,53531,53797
19,Спорт,64413,64413,64413,57894,64413
22,Экономика,79528,79528,79528,76423,79528
12,Мир,136621,136621,136620,136621,136621


# Подсчет слов и их статистика

In [33]:
dr = pd.read_csv(counted, encoding='windows-1251')

In [34]:
dr[:1000]

,0,1
0,год,1330489
1,в,1279151
2,который,929152
3,россия,718121
4,весь,564682
...,...,...
995,евросоюз,21266
996,дальнейший,21229
997,и,21182
998,благодаря,21177


In [46]:
# очистим от мусора
dr['new'] = dr['0'].apply(lambda x: x if str(x) not in rus_stopwords else 0)
dr = dr.loc[dr['new'] != 0]
dr = dr.drop(columns=['new'])

In [47]:
dr[:1000]

,0,1
0,год,1330489
3,россия,718121
6,сообщать,460883
7,также,445267
11,человек,371259
...,...,...
1047,сектор,20246
1048,родственник,20237
1049,разрешение,20231
1050,джон,20196


## Статистика по словам в датасете.
Полная статистика по употребляемым словам в датасете приведена в графике ниже.

In [48]:
trace = go.Scatter(x=dr['0'][:100], y=dr['1'][:100])

datas = [trace]
layout = go.Layout(barmode = "group", title='Words counting info',
                   xaxis= dict(title= "Quantity of words",ticklen= 5,zeroline= False), 
                   yaxis= dict(title= "Words",ticklen= 5,zeroline= False))
fig = go.Figure(data = datas, layout = layout)

offline.iplot(fig, validate=False)

In [50]:
dr.head(15) # top words in articles

,0,1
0,год,1330489
3,россия,718121
6,сообщать,460883
7,также,445267
11,человек,371259
12,время,368943
13,заявить,347297
15,российский,340010
16,компания,334833
17,стать,322794
